In [7]:
from dotenv import load_dotenv 
import os

In [8]:
load_dotenv()

False

In [9]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [14]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

# Agent A
agent_a = ChatOpenAI(model="gpt-4o").bind(system="คุณคือ Agent A ช่วยวิเคราะห์ข้อมูล")

# Agent B
agent_b = ChatOpenAI(model="gpt-4o").bind(system="คุณคือ Agent B ช่วยสรุปข้อมูล")

# Node Function
def agent_a_func(state):
    msg = state["messages"][-1]
    response = agent_a.invoke([msg])
    return {"messages": state["messages"] + [response]}

def agent_b_func(state):
    msg = state["messages"][-1]
    response = agent_b.invoke([msg])
    return {"messages": state["messages"] + [response]}

# Define Graph
builder = StateGraph()

# States
builder.add_node("agent_a", RunnableLambda(agent_a_func))
builder.add_node("agent_b", RunnableLambda(agent_b_func))

# Edges
builder.set_entry_point("agent_a")
builder.add_edge("agent_a", "agent_b")
builder.add_edge("agent_b", END)

graph = builder.compile()


ValueError: Must provide state_schema or input and output

In [12]:
output = graph.invoke({"messages": [HumanMessage(content="อธิบายผลลัพธ์ของโมเดล AI")]})
for m in output["messages"]:
    print(m.content)


NameError: name 'graph' is not defined